In [33]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [34]:
url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [35]:
response=requests.get(url)

In [36]:
soup=bs(response.text,'lxml')

In [ ]:
print(soup.prettify())

In [45]:
title = soup.find('div',class_="content_title").text

In [46]:
print(title)



NASA to Host Virtual Briefing on February Perseverance Mars Rover Landing




In [ ]:
title=results[0].a.text
print(title)

In [53]:
news_p = soup.find_all('div', class_="rollover_description_inner")[0].text

In [54]:
print(news_p)


NASA leadership and members of the mission will discuss the agency’s latest rover, which touches down on the Red Planet on Feb. 18.



AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url_2='https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url_2)
html_2=browser.html

In [ ]:
soup_2=bs(html_2,"html.parser")


In [ ]:
image_url=soup_2.find_all('img')[1]["src"]


In [ ]:
featured_image='https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/' + image_url

In [ ]:
url_3="https://space-facts.com/mars/"
tables=pd.read_html(url_3)
tables

In [ ]:
df=tables[0]
df.columns=['Category','Mars']

In [ ]:
df.set_index("Category")

In [ ]:
df.to_html("mars_facts.html",border=1)

In [ ]:
hemisphere_image_urls=[]
click_lst=['Cerberus','Schiaparelli','Syrtis','Valles']

In [ ]:
url_4='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
for x in range(4):
    browser.visit(url_4)
    browser.links.find_by_partial_text(click_lst[x]).click()
    html_3=browser.html
    soup_3=bs(html_3,'html.parser')
    titles=soup_3.find('h2',class_="title").text
    img_url=soup_3.find_all('img',class_="wide-image")[0]['src']
    hemisphere_image_urls.append({"title":title,"img_url":img_url})
    
    


In [ ]:
browser.quit()